In [1]:
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim

In [2]:
path = "C:/py/covid19/data/uncover/OpenTable/"
df = pd.read_csv(path + 'restaurant-performance-city.csv')

In [3]:
df.head()

,region_type,city,state,country,percent_yoy_change,date
0,cities,Atlanta,Georgia,United States,6,2020-02-18
1,cities,Atlanta,Georgia,United States,9,2020-02-19
2,cities,Atlanta,Georgia,United States,-5,2020-02-20
3,cities,Atlanta,Georgia,United States,-5,2020-02-21
4,cities,Atlanta,Georgia,United States,-3,2020-02-22


In [4]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("Atlanta, Georgia")

In [5]:
#get posible combinations of city and state
#temp_df = df[['city', 'state']].groupby(by=['city','state']).count()
cities = df.city.unique().tolist()

lat = []
lon = []

for city in cities:
    # search location name to get lat and lon
    location = geolocator.geocode( city )
    lat.append(location.latitude)
    lon.append(location.longitude)

In [6]:
df_loc = pd.DataFrame(cities)
df_loc.columns = ['city']
df_loc['lat'] = lat
df_loc['lon'] = lon

df_loc.head()

,city,lat,lon
0,Atlanta,33.749099,-84.390185
1,Austin,30.271129,-97.743700
2,Baltimore,39.290882,-76.610759
3,Boston,42.360253,-71.058291
4,Calgary,51.053423,-114.062589


In [7]:
lat = []
lon = []
for i, row in df.iterrows():
    lat.append(df_loc[df_loc['city'] == row['city']]['lat'].values[0])
    lon.append(df_loc[df_loc['city'] == row['city']]['lon'].values[0])

In [8]:
df['lat'] = lat
df['lon'] = lon
df['date'] = pd.to_datetime(df['date']).dt.date

In [9]:
df.head()

,region_type,city,state,country,percent_yoy_change,date,lat,lon
0,cities,Atlanta,Georgia,United States,6,2020-02-18,33.749099,-84.390185
1,cities,Atlanta,Georgia,United States,9,2020-02-19,33.749099,-84.390185
2,cities,Atlanta,Georgia,United States,-5,2020-02-20,33.749099,-84.390185
3,cities,Atlanta,Georgia,United States,-5,2020-02-21,33.749099,-84.390185
4,cities,Atlanta,Georgia,United States,-3,2020-02-22,33.749099,-84.390185


In [10]:
dates = df.date.unique()

In [11]:
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world[~world.name.isin(["Antarctica", "Fr. S. Antarctic Lands"])]

In [12]:
def plot_map(df, i):
    fig, ax = plt.subplots(1, figsize=(20,10), facecolor='lightblue')

    # plot map
    world.plot(ax=ax, color='gray')
    
    myMap = df[df['date'] == i]
    # plot infected locations
    plt.scatter(x=myMap['lon'], y=myMap['lat'], s=200, c=myMap['percent_yoy_change'], 
                cmap='viridis', vmin=-100, vmax=20)
    
    plt.text(-60, 90, str(i)+'\nRestaurant Reservations YOY change', color = 'black', fontsize = 50, ha='center')
    
    ax.set_xlim(-170, 30)
    ax.set_ylim(0, 80)
    
    # set an axis for the color bar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.05)
    # color bar
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=-100, vmax=20))
    sm._A = []
    cbar = fig.colorbar(sm, cax=cax)
    #cbar.ax.set_yticklabels(['-100%', '-75%', '-50%', '-25%', '0%', '25%', '50%', '75%', '100%'])
    cbar.ax.set_yticklabels(['-100%', '-80%', '-60%', '-40%', '-20%', '0%', '20%'])
    
    cbar.ax.tick_params(labelsize=18)
    
        
    plt.subplots_adjust(left=0, right=0.95, top=1, bottom=-0.2)
    # hide map axis
    ax.axis('off')
    
    plt.savefig('pics/reserve/restaurant'+str(i)+'.png', facecolor='lightblue', edgecolor='none')
    plt.close()
    
#plot_map(df, dates[1])

In [13]:
filenames = []
for date in dates:
    plot_map(df, date)
    filenames.append('pics/reserve/restaurant'+str(date)+'.png')

In [14]:

# build a .gif with the pictures
def build_gif(filenames, title, duration=1):
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))

    imageio.mimwrite((title + '.gif'), images, duration=duration)

In [15]:
build_gif(filenames, 'restaurants', 0.5)